In [40]:
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

## Step 1 - Scraping
### NASA Mars News
Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [41]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tang\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [42]:
url = "https://redplanetscience.com"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")


In [43]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[0].text
# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
print(16*("-"))
print(news_p)

NASA Establishes Board to Initially Review Mars Sample Return Plans
----------------
The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.


### JPL Mars Space Images - Featured Image
* Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

In [46]:
jpl_url = "https://spaceimages-mars.com/"
browser.visit(jpl_url)
html = browser.html
soup = bs(html, "html.parser")
#Find featured image
featured_image_text = soup.find('h1', class_='media_feature_title').text.strip()
featured_image_url = browser.find_by_tag('img[class="headerimage fade-in"]')[0]["src"]
print(featured_image_url)
print(featured_image_text)


https://spaceimages-mars.com/image/featured/mars3.jpg
Dusty Space Cloud


## Mars Facts
* Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [45]:
facts_url = "https://galaxyfacts-mars.com"
browser.visit(facts_url)
facts_table = pd.read_html(facts_url, header=0, index_col=0)
trimmed_table = facts_table[0]
trimmed_table = trimmed_table.to_html()
trimmed_table.replace('\n','')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Mars - Earth Comparison</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'



### Mars Hemispheres

* Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [48]:
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)
html = browser.html
soup = bs(html, "html.parser")
mars_hems = soup.find('div',class_='collapsible results')
mars_item = mars_hems.find_all('div',class_='item')
hemisphere_image_urls=[]

for i in mars_item:
    #Description
    hemisphere = i.find('div', class_="description")
    title = hemisphere.h3.text

    #Image links
    hemisphere_link = hemisphere.a["href"]
    browser.visit(hemi_url + hemisphere_link)
    
    html = browser.html
    soup = bs(html, "html.parser")
   

    image_link = soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']
   
    #Dictionary
    hemisphere_image_urls.append({"title": title, "img_url": (hemi_url + image_url)})
    
print(hemisphere_image_urls)
browser.quit() 

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]
